<a href="https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kohya Trainer V9 - VRAM 12GB
### The Best Way for People Without Good GPUs to Fine-Tune the Stable Diffusion Model

This notebook has been adapted for use in Google Colab based on the [Kohya Guide](https://note.com/kohya_ss/n/nbf7ce8d80f29#c9d7ee61-5779-4436-b4e6-9053741c46bb). </br>
This notebook was adapted by [Linaqruf](https://github.com/Linaqruf)</br>
You can find the latest update to the notebook [here](https://github.com/Linaqruf/kohya-trainer/blob/main/kohya-trainer.ipynb).


# Install Kohya Trainer

In [ ]:
#@title Installing Dependencies
%cd /workspace/kohya-trainer

import os

def install_dependencies():
  #@markdown This will install required Python packages
  !pip install --upgrade -r script/requirements.txt
  !pip install -U gallery-dl
  !pip install huggingface_hub

  # Install WD1.4 Tagger dependencies
  !pip install tensorflow
  
  Install_xformers = False #@param {'type':'boolean'}
  
  if Install_xformers:
    !pip install -U -I --no-deps https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
  else:
    pass

  # Install BLIP dependencies
  !pip install timm==0.4.12
  !pip install fairscale==0.4.4

  !apt-get update && apt-get install -y libgl1 libglib2.0-0

# Install dependencies
install_dependencies()

#@markdown After Accelerate updated its version to 0.15.0, you can't manually input the config using
#@markdown `!accelerate config` in Google Colab. Instead, a `config.yaml` file will be generated by
#@markdown the `write_basic_config()` function. You can find the file [here](/workspace/kohya-trainer/accelerate_config/config.yaml) after installation.
#@markdown if you want to modify it.

from accelerate.utils import write_basic_config
accelerate_config = "/workspace/kohya-trainer/accelerate_config/config.yaml"
write_basic_config(save_location = accelerate_config) # Write a config file

# Prepare Cloud Storage (Huggingface/GDrive)

In [ ]:
#@title Login to Huggingface hub

#@markdown ## Instructions:
#@markdown 1. Of course, you need a Huggingface account first.
#@markdown 2. To create a huggingface token, go to `Profile > Access Tokens > New Token > Create a new access token` with the `Write` role.

%cd /workspace/kohya-trainer

from huggingface_hub import notebook_login
notebook_login()



# Collecting datasets

You can either upload your datasets to this notebook or use the image scraper below to bulk download images from Danbooru.

If you want to use your own datasets, you can upload to colab `local files`.


In [ ]:
#@title Define Train Data
#@markdown Define where your train data will be located. This cell will also create a folder based on your input. 
#@markdown This folder will be used as the target folder for scraping, tagging, bucketing, and training in the next cell.

import os

train_data_dir = "/content/kohya-trainer/train_data" #@param {'type' : 'string'}

if not os.path.exists(train_data_dir):
    os.makedirs(train_data_dir)
else:
    print(f"{train_data_dir} already exists\n")

print(f"Your train data directory : {train_data_dir}")

In [ ]:
#@title Booru Scraper
#@markdown Use gallery-dl to scrape images from a booru site using the specified tags

%cd /workspace

# Set configuration options
booru = "Gelbooru" #@param ["", "Danbooru", "Gelbooru"]
tag1 = "hito_komoru" #@param {type: "string"}
tag2 = "" #@param {type: "string"}

# Construct the search query
if tag2 != "":
  tags = tag1 + "+" + tag2
else:
  tags = tag1

# Scrape images from the specified booru site using the given tags
if booru.lower() == "danbooru":
  !gallery-dl "https://danbooru.donmai.us/posts?tags={tags}" -D {train_data_dir}
elif booru.lower() == "gelbooru":
  !gallery-dl "https://gelbooru.com/index.php?page=post&s=list&tags={tags}" -D {train_data_dir}
else:
  print(f"Unknown booru site: {booru}")


In [ ]:
#@title Datasets cleaner
#@markdown This will delete unnecessary files and unsupported media like `.mp4`, `.webm`, and `.gif`

%cd /workspace

import os

train_data_dir = "/workspace/kohya-trainer/train_data" #@param {'type' : 'string'}

test = os.listdir(train_data_dir)

#@markdown I recommend to `keep_metadata` especially if you're doing resume training and you have metadata and bucket latents file from previous training like `.npz`, `.txt`, and `.caption`.
keep_metadata = True #@param {'type':'boolean'}

# List of supported file types
if keep_metadata == True:
  supported_types = [".jpg", ".jpeg", ".png", ".caption", ".npz", ".txt"]
else:
  supported_types = [".jpg", ".jpeg", ".png"]

# Iterate over all files in the directory
for item in test:
    # Extract the file extension from the file name
    file_ext = os.path.splitext(item)[1]
    # If the file extension is not in the list of supported types, delete the file
    if file_ext not in supported_types:
        # Print a message indicating the name of the file being deleted
        print(f"Deleting file {item} from {train_data_dir}")
        # Delete the file
        os.remove(os.path.join(train_data_dir, item))


# Dataset Labeling

In [ ]:
#@title Start BLIP Captioning
%cd /workspace/kohya-trainer

import shutil
import os

def clone_and_prepare_spaces():
    """
    Clones the Spaces repository, downloads the BLIP model weights, and moves the make_captions.py script to the BLIP directory.
    """
    # Constants
    BLIP_WEIGHT_SOURCE_URL = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth'
    BLIP_WEIGHT_DESTINATION_PATH = '/workspace/kohya-trainer/BLIP/model_large_caption.pth'
    MAKE_CAPTION_SOURCE_PATH = '/workspace/kohya-trainer/diffuser_fine_tuning/make_captions.py'
    MAKE_CAPTION_DESTINATION_PATH = '/workspace/kohya-trainer/BLIP/make_captions.py'

    # Install Git LFS
    !git lfs install

    # Clone the Spaces repository
    !git clone https://huggingface.co/spaces/Salesforce/BLIP

    # Download the BLIP model weights
    !wget -c {BLIP_WEIGHT_SOURCE_URL} -O {BLIP_WEIGHT_DESTINATION_PATH}

    # Move the make_captions.py script to the BLIP directory
    if os.path.exists(MAKE_CAPTION_SOURCE_PATH):
        shutil.move(MAKE_CAPTION_SOURCE_PATH, MAKE_CAPTION_DESTINATION_PATH)
    else:
        pass

# Clone and prepare Spaces
clone_and_prepare_spaces()

%cd /workspace/kohya-trainer/BLIP

#@markdown ### Define parameter:
batch_size = 8 #@param {'type':'integer'}
caption_extension = ".caption" #@param [".txt",".caption"]
caption_weights = "model_large_caption.pth"

!python make_captions.py \
  {train_data_dir} \
  {caption_weights} \
  --batch_size {batch_size} \
  --caption_extension {caption_extension}

In [ ]:
#@title Start WD 1.4 Tagger

# Change the working directory to the weight directory
%cd /workspace/kohya-trainer/diffuser_fine_tuning

#@markdown ### Define parameter:
batch_size = 4 #@param {'type':'integer'}
caption_extension = ".txt" #@param [".txt",".caption"]

!python tag_images_by_wd14_tagger.py \
  {train_data_dir} \
  --batch_size {batch_size} \
  --caption_extension .txt

In [ ]:
#@title Create meta_clean.json 
# Change the working directory
%cd /workspace/kohya-trainer/diffuser_fine_tuning

#@markdown ### Define Parameters
meta_cap_dd = "/workspace/kohya-trainer/meta_cap_dd.json" 
meta_cap = "/workspace/kohya-trainer/meta_cap.json" 
meta_clean = "/workspace/kohya-trainer/meta_clean.json" #@param {'type':'string'}

# Check if the train_data_dir exists and is a directory
if os.path.isdir(train_data_dir):
  # Check if there are any .caption files in the train_data_dir
  if any(file.endswith('.caption') for file in os.listdir(train_data_dir)):
    # Create meta_cap.json from captions
    !python merge_captions_to_metadata.py \
      {train_data_dir} \
      {meta_cap}

  # Check if there are any .txtn files in the train_data_dir
  if any(file.endswith('.txt') for file in os.listdir(train_data_dir)):
    # Create meta_cap_dd.json from tags
    !python merge_dd_tags_to_metadata.py \
      {train_data_dir} \
      {meta_cap_dd}
else:
  print("train_data_dir does not exist or is not a directory.")

# Merge meta_cap.json to meta_cap_dd.json
if os.path.exists(meta_cap) and os.path.exists(meta_cap_dd):
  !python merge_dd_tags_to_metadata.py \
    {train_data_dir} \
    --in_json {meta_cap} \
    {meta_cap_dd}

# Clean meta_cap_dd.json and store it to meta_clean.json
if os.path.exists(meta_cap_dd):
  # Clean captions and tags in meta_cap_dd.json and store the result in meta_clean.json
  !python clean_captions_and_tags.py \
    {meta_cap_dd} \
    {meta_clean}
elif os.path.exists(meta_cap):
  # If meta_cap_dd.json does not exist, clean meta_cap.json and store the result in meta_clean.json
  !python clean_captions_and_tags.py \
    {meta_cap} \
    {meta_clean}


# Prepare Training

In [ ]:
#@title Install Pre-trained Model 
%cd /workspace/kohya-trainer
import os

# Check if directory exists
if not os.path.exists('checkpoint'):
  # Create directory if it doesn't exist
  os.makedirs('checkpoint')

#@title Install Pre-trained Model
pretrained_model_url = "https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned-fp16.ckpt"

!gdown {pretrained_model_url} --fuzzy -O /workspace/kohya-trainer/checkpoint/pretrained.ckpt


In [ ]:
#@title Aspect Ratio Bucketing

# Change working directory
%cd /workspace/kohya-trainer/diffuser_fine_tuning

#@markdown ### Define parameters

model_dir = "/workspace/kohya-trainer/checkpoint/pretrained.ckpt" #@param {'type' : 'string'} 
batch_size = 4 #@param {'type':'integer'}
max_resolution = "512,512" #@param ["512,512", "768,768"] {allow-input: false}
mixed_precision = "no" #@param ["no", "fp16", "bf16"] {allow-input: false}
meta_clean = "/workspace/kohya-trainer/meta_clean.json"
meta_lat = "/workspace/kohya-trainer/meta_lat.json"

# Run script to prepare buckets and latents
!python prepare_buckets_latents.py \
  {train_data_dir} \
  {meta_clean} \
  {meta_lat} \
  {model_dir} \
  --batch_size {batch_size} \
  --max_resolution {max_resolution} \
  --mixed_precision {mixed_precision}



  

# Start Training



In [ ]:
#@title Define Important folder
import os

pre_trained_model_path ="/workspace/kohya-trainer/checkpoint/pretrained.ckpt" #@param {'type':'string'}
meta_lat_json_dir = "/workspace/kohya-trainer/meta_lat.json" #@param {'type':'string'}
train_data_dir = "/workspace/kohya-trainer/train_data" #@param {'type':'string'}
output_dir ="/workspace/kohya-trainer/fine-tuned" #@param {'type':'string'}
resume_path = "" #@param {'type':'string'}

# List of important folder paths
folder_paths = [
    pre_trained_model_path,
    meta_lat_json_dir,
    train_data_dir,
    output_dir,
    resume_path
]

# Check if each folder exists
for folder_path in folder_paths:
    if folder_path:
        try:
            if os.path.exists(folder_path):
                print(f'{folder_path} can be used, located at {os.path.dirname(folder_path)}')
            else:
                pass
        except:
            print(f'An error occurred while checking if {folder_path} exists')
    else:
        print('Empty folder path')


In [ ]:
#@title Training begin
#@markdown ### Define Parameters
num_cpu_threads_per_process = 8 #@param {'type':'integer'}
save_state = True #@param {'type':'boolean'}
train_batch_size = 2  #@param {type: "slider", min: 1, max: 10}
learning_rate ="2e-6" #@param {'type':'string'}
max_train_steps = 1000 #@param {'type':'integer'}
train_text_encoder = False #@param {'type':'boolean'}
lr_scheduler = "constant" #@param  ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"] {allow-input: false}
max_token_length = "225" #@param  ["150", "225"] {allow-input: false}
clip_skip = 1 #@param {type: "slider", min: 1, max: 10}
mixed_precision = "fp16" #@param ["no","fp16","bf16"] {allow-input: false}
save_precision = "fp16" #@param ["None","float", "fp16", "bf16"] {allow-input: false}
save_every_n_epochs = 200 #@param {'type':'integer'}
gradient_accumulation_steps = 1 #@param {type: "slider", min: 1, max: 10}
#@markdown ### Log And Debug
log_prefix = "fine-tune-style1" #@param {'type':'string'}
logs_dst = "/workspace/kohya-trainer/logs" #@param {'type':'string'}
debug_mode = False #@param {'type':'boolean'}

if save_state == True:
  sv_state = "--save_state"
else:
  sv_state = ""

if resume_path != "":
  rs_state = "--resume " + str(resume_path)
else:
  rs_state = ""

if save_every_n_epochs != 0 :
  save_epoch = "--save_every_n_epochs" + "=" + "{}".format(save_every_n_epochs)
else:
  save_epoch = ""

if save_precision != "None":
  sv_precision = "--save_precision=" + str(save_precision)
else :
  sv_precision = ""

if debug_mode == True:
  debug_dataset = "--debug_dataset"
else:
  debug_dataset = ""

if train_text_encoder == True:
  text_encoder = "--train_text_encoder"
else:
  text_encoder = ""


%cd /workspace/kohya-trainer/diffuser_fine_tuning
!accelerate launch \
  --config_file /workspace/kohya-trainer/accelerate_config/config.yaml \
  --num_cpu_threads_per_process {num_cpu_threads_per_process} \
  fine_tune.py \
  --pretrained_model_name_or_path={pre_trained_model_path} \
  --in_json {meta_lat_json_dir} \
  --train_data_dir={train_data_dir} \
  --output_dir={output_dir} \
  --shuffle_caption \
  --train_batch_size={train_batch_size} \
  --learning_rate={learning_rate} \
  --lr_scheduler={lr_scheduler} \
  --max_token_length={max_token_length} \
  --clip_skip={clip_skip} \
  --mixed_precision={mixed_precision} \
  --max_train_steps={max_train_steps} \
  --use_8bit_adam \
  --xformers \
  --gradient_checkpointing \
  --gradient_accumulation_steps {gradient_accumulation_steps} \
  {text_encoder} \
  {sv_state} \
  {rs_state} \
  {save_epoch} \
  {sv_precision} \
  {debug_dataset} \
  --logging_dir={logs_dst} \
  --log_prefix {log_prefix}


# Miscellaneous

In [ ]:
%cd /workspace/kohya-trainer/convert_diffusers20_original_sd

#@title Convert Weight to Diffusers or `.ckpt/.safetensors` (Optional)
#@markdown ## Define weight path
weight = "/workspace/kohya-trainer/fine-tuned/model.ckpt" #@param {'type': 'string'}
weight_dir = os.path.dirname(weight)
convert = "diffusers_to_ckpt_safetensors" #@param ["diffusers_to_ckpt_safetensors", "ckpt_safetensors_to_diffusers"] {'allow-input': false}

#@markdown ## Conversion Config
#@markdown
#@markdown ### Diffusers to `.ckpt/.safetensors`
use_safetensors = True #@param {'type': 'boolean'}

if use_safetensors:
    checkpoint = str(weight_dir)+"/model.safetensors"
else:
    checkpoint = str(weight_dir)+"/model.ckpt"

save_precision = "--float" #@param ["--fp16","--bf16","--float"] {'allow-input': false}

#@markdown ### `.ckpt/.safetensors` to Diffusers
#@markdown is your model v1 or v2 based Stable Diffusion Model
version = "--v1" #@param ["--v1","--v2"] {'allow-input': false}
diffusers = str(weight_dir)+"/diffusers_model"

#@markdown Add reference model to get scheduler, optimizer, and tokenizer, because `.ckpt/.safetensors` didn't have one.
reference_model ="/workspace/kohya-trainer/checkpoint/pretrained.ckpt" #@param {'type': 'string'}

if convert == "diffusers_to_ckpt_safetensors":
    if not weight.endswith(".ckpt") or weight.endswith(".safetensors"):
        !python convert_diffusers20_original_sd.py \
            {weight} \
            {checkpoint} \
            {save_precision}

else:    
    !python convert_diffusers20_original_sd.py \
        {weight} \
        {diffusers} \
        {version} \
        --reference_model {reference_model} 


In [ ]:
#@title Model Pruner (Optional)

##Lopho

#@markdown Do you want to Prune a model?
%cd /workspace/ 

# Use a more descriptive variable name
prune = True #@param {'type':'boolean'}

# Add a checkbox to enable/disable the `--fp16` argument
fp16 = False #@param {'type':'boolean'}

# Add a checkbox to enable/disable the `--ema` argument
ema = False #@param {'type':'boolean'}

# Add a checkbox to enable/disable the `--no-clip` argument
no_clip = False #@param {'type':'boolean'}

# Add a checkbox to enable/disable the `--no-vae` argument
no_vae = False #@param {'type':'boolean'}

# Use a more descriptive variable name
input = "/workspace/kohya-trainer/fine-tuned/model.ckpt" #@param {'type' : 'string'}

# Use a more descriptive variable name
output = "/workspace/kohya-trainer/fine-tuned/model-pruned.ckpt" #@param {'type' : 'string'}

if prune:
  import os
  if os.path.isfile('/workspace/prune.py'):
    pass
  else:
    # Add a comment to explain what the code is doing
    # Download the pruning script if it doesn't already exist
    !wget https://raw.githubusercontent.com/lopho/stable-diffusion-prune/main/prune.py


# Add a comment to explain what the code is doing
# Run the pruning script with the specified arguments
!python3 prune.py {input} {output} {'--fp16' if fp16 else ''} {'--ema' if ema else ''} {'--no-clip' if no_clip else ''} {'--no-vae' if no_vae else ''}


In [ ]:
#@title Visualize loss graph (Optional)
%cd /workspace/kohya-trainer
%load_ext tensorboard
%tensorboard --logdir {logs_dst}

## Commit trained model to Huggingface

### To Commit models:
1. Create a huggingface repository for your model.
2. Clone your model to this Colab session.
3. Move the necessary files to your repository to save your trained model to huggingface. These files are located in `fine-tuned` folder:
   - `epoch-nnnnn.ckpt` and/or
   - `last.ckpt`
4. Commit your model to huggingface.

### To Commit datasets:
1. Create a huggingface repository for your datasets.
2. Clone your datasets to this Colab session.
3. Move the necessary files to your repository so that you can resume training without rebuilding your dataset with this notebook:
  - The `train_data` folder.
  - The `meta_lat.json` file.
  - The `last-state` folder.
4. Commit your datasets to huggingface.



In [ ]:
#@title Clone Model or Datasets

#@markdown Type of item to clone (model or dataset)
type_of_item = "model" #@param ["model", "dataset"]

#@markdown Install or uninstall git lfs
install_git_lfs = False #@param {'type':'boolean'}

%cd /workspace
username = "your-huggingface-username" #@param {'type': 'string'}
model_repo = "your-huggingface-model-repo" #@param {'type': 'string'}
datasets_repo = "your-huggingface-datasets-repo" #@param {'type': 'string'}

if type_of_item == "model":
  Repository_url = f"https://huggingface.co/{username}/{model_repo}"
elif type_of_item == "dataset":
  Repository_url = f"https://huggingface.co/datasets/{username}/{datasets_repo}"

if install_git_lfs:
  !git lfs install
else:
  !git lfs uninstall
!git clone {Repository_url}


In [ ]:
#@title Commit Model or Datasets to Huggingface

#@markdown Type of item to commit (model or dataset)
type_of_item = "model" #@param ["model", "dataset"]

%cd /workspace
#@markdown Go to your model or dataset path
item_path = "your-cloned-model-or-datasets-repo" #@param {'type': 'string'}

#@markdown #Git Commit

#@markdown Set **git commit identity**
email = "your-email" #@param {'type': 'string'}
name = "your-username" #@param {'type': 'string'}
#@markdown Set **commit message**
commit_m = "feat: upload 6 epochs model" #@param {'type': 'string'}

%cd {item_path}
!git lfs install
!huggingface-cli lfs-enable-largefiles .
!git add .
!git lfs help smudge
!git config --global user.email "{email}"
!git config --global user.name "{name}"
!git commit -m "{commit_m}"
!git push
